# Retinaface로 bbox 정보 추출

In [ ]:
pip install onnxruntime-gpu #그냥 onnxruntime말고 onnxruntime-gpu로 설치해야 cuda 동작가능

pip install insightface

pip install opencv-python

[주의] <br>
https://forums.developer.nvidia.com/t/could-not-load-library-cudnn-cnn-infer64-8-dll-error-code-193/218437/16<br>
여기서 zlib123dllx64.zip 파일 다운 받은 후에 압축 풀어서 zlibwapi.dll파일을  c:/program files/nvidia gpu computin toolkit/cuda/v11.6/bin폴더에 복사해서 넣어줘야 cuda 동작됨. (v11.6은 우리가 설치한 버전)

In [1]:
import cv2
import os
import json
import insightface
from insightface.app import FaceAnalysis

# FaceAnalysis 객체 초기화
app = FaceAnalysis(providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640)) #ctx_id=0으로 설정해야 0번째 gpu사용

input_folder = 'omg_1min'  # 입력 이미지가 있는 폴더
bbox_folder = 'bbox_data'  # bbox 정보를 저장할 폴더

# bbox 폴더가 없으면 생성
if not os.path.exists(bbox_folder):
    os.makedirs(bbox_folder)

# 입력 폴더에서 이미지 파일 목록 가져오기
image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg') or f.endswith('.png')]

# 각 이미지에 대한 처리 및 bbox 정보 저장
for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    img = cv2.imread(image_path)
    
    # 얼굴 감지
    faces = app.get(img)
    
    # 감지된 얼굴에 대한 bbox 정보 저장
    bboxes = [face.bbox.astype(int).tolist() for face in faces]
    
    # bbox 정보 저장
    bbox_file_path = os.path.join(bbox_folder, image_file.replace('.jpg', '.json'))
    with open(bbox_file_path, 'w') as f:
        json.dump({"bbox": bboxes}, f)

print("모든 이미지에 대한 bbox 정보 저장 완료")


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'cudnn_conv_algo_search': 'EXHAUSTIVE', 'device_id': '0', 'has_user_compute_stream': '0', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_external_alloc': '0', 'gpu_mem_limit': '18446744073709551615', 'enable_cuda_graph': '0', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'cudnn_conv_use_max_workspace': '1', 'tunable_op_enable': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'prefer_nhwc': '0', 'use_ep_level_unified_stream': '0'}, 'CPUExecutionProvider': {}}
find model: C:\Users\오민준/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'cudnn_conv_algo_search': 'EXHAUSTIVE', 'devic

C:\Anaconda3\envs\nanodet\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


모든 이미지에 대한 bbox 정보 저장 완료
